<a href="https://colab.research.google.com/github/ciphenseadmin/partial-face-recognition/blob/master/Keras_Facenet_Finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files
from PIL import Image
import tensorflow as tf
import numpy as np
from scipy import spatial

In [0]:
from keras.models import load_model
model = load_model("/content/drive/My Drive/keras_facenet/facenet_keras.h5" )

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
# model.layers.pop()
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [0]:
from keras.layers import Flatten,Dense
from keras import Model

In [0]:
# model = Model(model.input,model.output)
last_layer = model.get_layer('Bottleneck_BatchNorm').output
out = Dense(217, activation='softmax', name='Classification_layer')(last_layer)

In [0]:
model = Model(model.input,out)

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
____________________________________________________________________________________________

In [0]:
layer_count = 0
for layer in model.layers:
	layer_count = layer_count+1
print(layer_count)


427


In [0]:
for l in range(layer_count-99):
	model.layers[l].trainable=False

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
____________________________________________________________________________________________

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],)

In [0]:
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras_vggface import utils
preprocess_input = preprocess_input
datagenerator = ImageDataGenerator(preprocessing_function = preprocess_input)

In [0]:
train_generator = datagenerator.flow_from_directory(
        '/content/drive/My Drive/SubsetLfwCropped/train',
        target_size=(160, 160),
        batch_size=32,
        class_mode='categorical',)

validation_generator = datagenerator.flow_from_directory(
        '/content/drive/My Drive/SubsetLfwCropped/test',
        target_size=(160, 160),
        class_mode='categorical')

hisory = model.fit_generator(
        train_generator,
        validation_data=validation_generator,
        validation_steps=1,epochs=50)

Found 3264 images belonging to 217 classes.
Found 1539 images belonging to 217 classes.
Epoch 1/50
102/102 [==============================] - 1551s 15s/step - loss: 3.0416 - accuracy: 0.4366 - val_loss: 3.6494 - val_accuracy: 0.2812
Epoch 2/50
102/102 [==============================] - 20s 199ms/step - loss: 1.3022 - accuracy: 0.7665 - val_loss: 3.5209 - val_accuracy: 0.4062
Epoch 3/50
102/102 [==============================] - 20s 197ms/step - loss: 0.5430 - accuracy: 0.9418 - val_loss: 3.0027 - val_accuracy: 0.4688
Epoch 4/50
102/102 [==============================] - 20s 198ms/step - loss: 0.1912 - accuracy: 0.9933 - val_loss: 2.6254 - val_accuracy: 0.5312
Epoch 5/50
102/102 [==============================] - 20s 197ms/step - loss: 0.0693 - accuracy: 0.9997 - val_loss: 2.4416 - val_accuracy: 0.5625
Epoch 6/50
102/102 [==============================] - 20s 197ms/step - loss: 0.0327 - accuracy: 0.9997 - val_loss: 1.6725 - val_accuracy: 0.6250
Epoch 7/50
102/102 [======================

In [0]:
cd "/content/drive/My Drive"

/content/drive/My Drive


In [0]:
model.save("50epoch_softmaxadded.h5")

In [0]:
_, acc = model.evaluate_generator(validation_generator, steps=len(validation_generator), verbose=0)

In [0]:
print(acc) #acc= 0.6751137375831604